In [3]:
import os
import random
import yaml

In [2]:
from typing import (
    Dict
)

In [7]:
def read_config(config: str) -> Dict[str,str]:
    """Reads the contents of a YAML (configuration) file to
    a dictionary.

    Arguemnts:
        config: Filename (and path) to configuration file.
    
    Returns:
        Dictionary of strings matched valued pairs.
    """
    config: str = os.path.abspath(
        os.path.realpath(config)
        )
    
    with open(config,"r") as file:
        cfg: Dict[str,str] = yaml.safe_load(file) 
    return cfg

In [8]:
cfg = 'config.people.yml'

In [9]:
read_config(config=cfg)

{'Person1': {'email': 'Person1@mail.com', 'exclude': 'Person2'},
 'Person2': {'email': 'Person2@mail.com'},
 'Person3': {'email': 'Person3@mail.com'},
 'Person4': {'email': 'Person4@mail.com'}}